# Projeto 12: Validação Cruzada - Rede Neural Convolucional MNIST

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 KB 4.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.model_selection import cross_val_score
import torchvision.transforms as transforms
from skorch import NeuralNetClassifier
torch.__version__

'1.13.1+cu116'

## Etapa 2: Base de dados

In [3]:
torch.manual_seed(123)

In [4]:
transform = transforms.ToTensor()

In [5]:
train = datasets.MNIST(root = '.', train = True, download=True, transform=transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



## Etapa 3: Construção do modelo

In [6]:
class classificador_torch(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3))
    self.conv2 = nn.Conv2d(32, 32, (3, 3))
    self.activation = nn.ReLU()
    self.bnorm = nn.BatchNorm2d(num_features=32)
    self.pool = nn.MaxPool2d(kernel_size = (2,2))
    self.flatten = nn.Flatten()

    self.linear1 = nn.Linear(in_features=32*5*5, out_features=128)
    self.linear2 = nn.Linear(128, 128)
    self.output = nn.Linear(128, 10)
    self.dropout = nn.Dropout(p = 0.2)

  def forward(self, X):
    X = X.view(-1, 1, 28, 28) ##### Tem que transformar as imagens no formato que as CNNs operam no PyTorch: (batch_size, n_dimensions, width, height) 
    X = self.pool(self.bnorm(self.activation(self.conv1(X))))
    X = self.pool(self.bnorm(self.activation(self.conv2(X))))
    X = self.flatten(X)

    X = self.dropout(self.activation(self.linear1(X)))
    X = self.dropout(self.activation(self.linear2(X)))
    X = self.output(X)

    return X

In [7]:
classificador_sklearn = NeuralNetClassifier(module = classificador_torch, 
                                            criterion = torch.nn.CrossEntropyLoss, 
                                            optimizer = torch.optim.Adam,
                                            max_epochs = 3,
                                            batch_size = 10,
                                            train_split = False)

## Etapa 4: Validação cruzada

In [8]:
##### Primeiro temos que mudar os tipos dos dados de X e y para corresponderem aos tipos dos pesos usados na rede.
X = train.data.type('torch.FloatTensor')
y = train.targets.type('torch.LongTensor')

In [9]:
resultados = cross_val_score(classificador_sklearn, X, y, cv = 5, scoring = 'accuracy')

  epoch    train_loss      dur
-------  ------------  -------
      1        0.4505  44.6001
      2        0.3710  41.1705
      3        0.3894  40.9916
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4111  39.1019
      2        0.3830  41.7234
      3        0.4002  41.9682
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4098  39.3986
      2        0.4048  41.1647
      3        0.3694  41.4661
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4074  38.9200
      2        0.3847  39.8384
      3        0.3736  41.5679
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4070  38.9668
      2        0.3812  39.7162
      3        0.3758  40.4642


In [10]:
media = resultados.mean()
desvio = resultados.std()

In [11]:
media, desvio

(0.7245166666666667, 0.11000458828814573)

In [12]:
resultados

array([0.72508333, 0.79816667, 0.51441667, 0.76325   , 0.82166667])